# SDEV 140 Fall 2023
# Evan Mitchell
# Rainbow Six Siege App
# Keeps track of current match round scores as well as
# Overall round win/loss for each operator

Classes used to maintain object states
Objects will be saved to .obj files using pickle

In [25]:
from breezypythongui import EasyFrame
import tkinter as tk
import pickle
from PIL import ImageTk,Image

In [26]:
class GameMatch:
    def __init__(self,username):
        self.wins = 0
        self.loses = 0
        self.user = username

In [27]:
class Operator:
    def __init__(self,name,wins=0,loses=0):
        self.name = name
        self.wins = wins
        self.loses = loses

Our MainFrame class will represent our main window with a button to start a new match

In [28]:
class MainFrame(EasyFrame):
    def __init__(self):
        EasyFrame.__init__(self,title="Game",height=500,width=500)
        self.config(bg="blue")

The Match class will handle the majority of our logic

In [29]:


class Match(EasyFrame):

    def __init__(self,starting_state):
        EasyFrame.__init__(self,title="Match",width=500,height=500)
        self.config(bg="#869eb9")

        self.starting_state = starting_state
        self.match = GameMatch("emitchell")
        self.img_label = self.addLabel(text="",row=3,column=1,sticky="NSEW")
        self.display = self.addTextField(text="",row=3,column=0,sticky="NSEW")
        
        ### Operator Setup ###

        self.attacking_ops = ['ace','amaru','ash','blackbeard','blitz','brava',
                              'buck','capitao','dokkaebi','finka','flores','fuze',
                              'glaz','gridlock','grim','hibana','iana','iq','jackal',
                              'kali','lion','maverick','montagne','nokk','nomad','osa',
                              'ram','sens','sledge','thatcher','thermite','twitch','ying',
                              'zero','zofia']
        self.defending_ops = ['alibi','aruni','azami','bandit','castle','caveira','clash',
                              'doc','echo','ela','fenrir','frost','goyo','jager','kaid',
                              'kapkan','lesion','maestro','melusi','mira','mozzie','mute',
                              'oryx','pulse','rook','smoke','solis','tachanka','thorn',
                              'thunderbird','tubarao','valkyrie','vigil','wamai','warden']
        
        self.op_name = ""
        self.current_op = None
        
        ### ROUND DATA ###
        self.round_lable = self.addTextField(text=f"{self.match.wins} : {self.match.loses}",row=0,column=0,columnspan=2,sticky="NSEW")
        ### BUTTONS ###
        self.addButton(text="Round Won",row=2,column=0,command=self.round_won,sticky="NSEW").config(bg="#64ad6a")
        self.addButton(text="Round Lost",row=2,column=1,command=self.round_lost,sticky="NSEW").config(bg="#dd3439")
        
        ### LIST BOXES ###
        self.attack_list = self.addListbox(row=1,column=0,height=3,listItemSelected=self.get_operator)
        self.defend_list = self.addListbox(row=1,column=1,height=3,listItemSelected=self.get_operator)

        #Populate the lists  for both attacking and defending operators
        for op in self.attacking_ops:
            self.attack_list.insert(tk.END,op)
        
        for op in self.defending_ops:
            self.defend_list.insert(tk.END,op)
    
        #disable list of operators depending on the starting state of the match
        if self.starting_state == "attack":
            self.defend_list['state'] = "disabled"
        else:
            self.attack_list['state'] = "disabled"

    
    def get_operator(self,index):
        #save the previous operators data
        if self.current_op != None:
            fp = open(f"{self.op_name}.obj","wb")
            pickle.dump(self.current_op,fp)
            fp.close()
            
        #self.op_name = self.prompterBox(title="Operator",promptString="Pick an operator",inputText="",fieldWidth=20).lower()
        if self.starting_state == "attack":
            self.op_name = self.attack_list.getSelectedItem()
        else:
            self.op_name = self.defend_list.getSelectedItem()
        #if the operator has been used before, read from the file
        try:
            fp = open(f"{self.op_name}.obj","rb")
            self.current_op = pickle.load(fp)
            fp.close()
        #no file exists so create a new Operator object and write it to a new .obj file
        except:
            self.current_op = Operator(self.op_name)
            fp  = open(f"{self.op_name}.obj","wb")
            pickle.dump(self.current_op,fp)
            fp.close()
            
        #add the operator icon to a label
        try:
            self.img = ImageTk.PhotoImage(Image.open(f"./r6/{self.op_name}.png").resize((150,150)))
        except:
            self.img = ImageTk.PhotoImage(Image.open("./r6/recruit_orange.png").resize((150,150)))
        self.img_label.configure(image=self.img)
        self.img_label.image = self.img
        
        #update the display to show new operator data
        self.update_display()

    def swap_sides(self):
        if self.starting_state == "attack":
            self.starting_state = "defend"
            self.attack_list['state'] = tk.DISABLED
            self.defend_list['state'] = tk.NORMAL
        else:
            self.starting_state = "attack"
            self.attack_list['state'] = tk.NORMAL
            self.defend_list['state'] = tk.DISABLED

    #Show the current operators all time win/loss
    def update_display(self):
        total_rounds_played = self.current_op.wins + self.current_op.loses
        if total_rounds_played > 0:
            self.display.setText(f"{self.current_op.name.capitalize()}\n Win Percentage: {self.current_op.wins/total_rounds_played:.2f}")
        else:
            self.display.setText(f"{self.current_op.name.capitalize()}\n Win Percentage: No Rounds Played")
    #Returns True when specific round requirements are met
    def check_win(self):
        if self.match.wins == 5 or (self.match.wins == 4 and self.match.loses < 3):
            return True
        return False
    
    #Returns True when specific round requirements are met
    def check_loss(self):
        if self.match.loses == 5 or (self.match.loses == 4 and self.match.wins < 3):
            return True
        return False
    
    #Update current rounds won and check for win condition
    def round_won(self):
        self.match.wins += 1
        self.current_op.wins += 1
        self.round_lable.setText(f"{self.match.wins} : {self.match.loses}")
        self.update_display()
        if self.check_win():
            print("You won the game!")
            self.destroy()
        
        if self.match.wins + self.match.loses == 3:
            self.swap_sides()
        
    #UPdate current rounds lost and check for lost condition
    def round_lost(self):
        self.match.loses += 1
        self.current_op.loses += 1
        print(self.match.loses)
        self.round_lable.setText(f"{self.match.wins} : {self.match.loses}")
        self.update_display()
        if self.check_loss():
            print("You lost the game!")
            self.destroy()
        if self.match.wins + self.match.loses == 3:
            self.swap_sides()

In [30]:

#Base Application Window
root = tk.Tk()

###Create an object from class MainFrame
main_window = MainFrame()
main_window.grid(row=0,column=0)

match_window = None

def start_match(side):
    global match_window
    match_window = Match(side)
    match_window.grid(row=0,column=0,sticky="NSEW")

def finish_match():
    global match_window
    match_window.grid_forget()
    match_window = Match("attack")

### Welcome Message ###
welcome_label = tk.Label(main_window,text="Select a starting Side")
welcome_label.grid(row=0,column=0,columnspan=4,sticky="NSEW")
### Main Window Buttons for handling the start of a match ###
start_attack_side = tk.Button(main_window,text="Start Attack",command=lambda:start_match("attack"))
start_defend_side = tk.Button(main_window,text="Start Defend",command=lambda:start_match("defend"))
start_attack_side.grid(row=1,column=0,columnspan=2,sticky="NSEW")
start_defend_side.grid(row=1,column=2,columnspan=2,sticky="NSEW")

root.mainloop()

To-Do:
    Handle overtime condition for swapping sides
    Sort attack and defend list in ASC order
    Find op images for the newest operators
    